In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

## Plot
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import matplotlib as plt

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Other
import re
import string
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data=pd.read_csv(r'/content/drive/My Drive/train (2).csv')



In [ ]:
data.isna().sum()
data.drop(columns=['qid'],inplace=True)

In [ ]:

y_final=np.array(data['target'])
data['question_text'][1]


'Do you have an adopted dog, how would you encourage people to adopt and not shop?'

In [ ]:
import nltk
from nltk.tokenize import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation
token=ToktokTokenizer()
punctuation
punct='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
lemma=WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def lemitizeWords(text):
    words=token.tokenize(text)
    listLemma=[]
    for w in words:
        x=lemma.lemmatize(w, pos="v")
        listLemma.append(x)
    return ' '.join(map(str, listLemma))

def stopWordsRemove(text):
    stop_words = set(stopwords.words("english"))
    words=token.tokenize(text)
    filtered = [w for w in words if not w in stop_words]
    return ' '.join(map(str, filtered))


def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [ ]:
# data['question_text'] = data['question_text'].apply(lambda x: clean_punct(x)) 

data['question_text'] = data['question_text'].apply(lambda x: x.lower()) 

data['question_text'] = data['question_text'].apply(remove_punctuations)


data['question_text'] = data['question_text'].apply(lambda x: lemitizeWords(x)) 
data['question_text'] = data['question_text'].apply(lambda x: stopWordsRemove(x)) 


In [ ]:
text=data['question_text'].tolist()
text
token=Tokenizer()
token.fit_on_texts(text)

In [ ]:
vocab_size=len(token.word_index)+1
vocab_size

noofwords= data['question_text'].apply(lambda x: len(x.split()))
maxlen=max(noofwords)

In [ ]:

tokenizer = Tokenizer(num_words= vocab_size)
tokenizer.fit_on_texts(data['question_text'])

sequences = tokenizer.texts_to_sequences(data['question_text'])
data = pad_sequences(sequences, maxlen)

In [ ]:
data.shape

(1306122, 152)

In [ ]:
embeddings_index = dict()
f = open("/content/drive/My Drive/glove.6B.100d.txt")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [ ]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    if index > vocab_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [ ]:
model_glove = Sequential()
model_glove.add(Embedding(vocab_size, 100, input_length=152, weights=[embedding_matrix], trainable=False))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(LSTM(100))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model_glove.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 152, 100)          23543600  
_________________________________________________________________
dropout_1 (Dropout)          (None, 152, 100)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 148, 64)           32064     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 37, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 23,641,765
Trainable params: 98,165
Non-trainable params: 23,543,600
_____________________________________

In [ ]:
# s=np.array(v['target'])

X_final.shape,y_final.shape

((1306122, 152), (1306122,))

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
X_final=np.array(data)
# y_final=np.array(v['target'])
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((875101, 152), (875101,), (431021, 152), (431021,))

In [ ]:
model_glove.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)


Train on 875101 samples, validate on 431021 samples
Epoch 1/10
875101/875101 [==============================] - 1597s 2ms/step - loss: 0.1327 - accuracy: 0.9494 - val_loss: 0.1195 - val_accuracy: 0.9540
Epoch 2/10
875101/875101 [==============================] - 1582s 2ms/step - loss: 0.1218 - accuracy: 0.9527 - val_loss: 0.1146 - val_accuracy: 0.9555
Epoch 3/10
875101/875101 [==============================] - 1578s 2ms/step - loss: 0.1181 - accuracy: 0.9538 - val_loss: 0.1161 - val_accuracy: 0.9553
Epoch 4/10
875101/875101 [==============================] - 1582s 2ms/step - loss: 0.1156 - accuracy: 0.9549 - val_loss: 0.1132 - val_accuracy: 0.9558
Epoch 5/10
875101/875101 [==============================] - 1588s 2ms/step - loss: 0.1140 - accuracy: 0.9555 - val_loss: 0.1132 - val_accuracy: 0.9558
Epoch 6/10
875101/875101 [==============================] - 1603s 2ms/step - loss: 0.1127 - accuracy: 0.9558 - val_loss: 0.1120 - val_accuracy: 0.9563
Epoch 7/10
875101/875101 [================

In [ ]:
y_pred=model_glove.predict_classes(X_test)


In [ ]:

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9559302214973284

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,y_pred)

array([[397845,   6899],
       [ 12096,  14181]])